# Desarroyo de Modelo de Machine Learning


### DataFrame que voy a utilizar para alimentar mi modelo de prediccion item x item. Analiza los tags de cada uno de los items y recomienda en base a ellos.

Para la implementacion del modelo utilizo 2 dataframes. Uno donde solo almaceno los mejores 2000 juegos (en base a su "score")y otro donde estan todos.

En el dataframe reducido se utilizará para entrenar el modelo de recomendacion. En base a este modelo entrenado existen dos posibles alternativas:

- Que el juego ingresado para que se haga la recomendacion esté en el modelo, en tal caso se realizará una recomendacion sin necesidad de utilziar computo practicamente, solo será una busqueda en por indice, ya que se utilizará un archivo .pkl donde estrá nuestro modelo entrenado(caso ideal para optimizar recursos)

- Que el juego ingresado para que se haga la recomendacion no esté en el modelo original, en tal caso se deberá realizar el procedimiento enterp, añadiendo ese juego con sus caracteristicas al df de donde se sacan las recomendaciones.

- Los dos Dataframes utilizados para este modelo se encuentran en el notebook Dataframes_API.ipynb

In [1]:
import pandas as pd
df_ML = pd.read_csv(r"data_api\datos_ML.csv")
df_ML_reduced = pd.read_csv(r"data_api\datos_ML_reduced.csv")

In [2]:
#importamos librerias, voy a utilizar CountVectorizer para poder vectorizar la informacion. 
#para esto usamos CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=45, stop_words="english")

Vectorizamos para pasar texto a un vector lo que nos permite haces comparaciones entre ellos y encontrar similitudes de coseno.

In [3]:
#toarray() para convertirlo en un array de numpy. seleccionamos solo las 2000 mejor puntuadas
vectors = cv.fit_transform(df_ML_reduced["tags"]).toarray()

In [ ]:
print(vectors)

In [4]:
#ahora vamos a aplicar el algoritmo de coseno de similitud
#importamos cosine_similarity y la aplicamos a nuestra data que vamos a usar para el modelo.
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)

In [5]:
#exportamos el modelo para despues no tener que volver a entrenarlo
import joblib
joblib.dump(similarity,'modelo_entrenado.pkl')

['modelo_entrenado.pkl']

In [6]:
similarity.shape

(2000, 2000)

In [7]:
#esta es la funcion con la que implementariamos el modelo, como vemo hacer unas recomendaciones bastantes acertadas sin caer en la reduncancia.
#dentro de los try y except no se puede llamar a variables externas a la funcion por o que me veo forzado a correr siempre la carga del modelo.
#basicamente aplica una busqueda por id al modelo.

#si no lo encuentra en primera instancia, pasa a crear el registro, añadirlo y realizar la prediccion.
def recommend(id_item):
    try:
        similarity = joblib.load('modelo_entrenado.pkl')
        idx = df_ML_reduced[df_ML_reduced["item_id"] == id_item].index[0]
        distances = similarity[idx]   
        jueguito = sorted(list(enumerate(distances)), reverse=True, key= lambda x:x[1])[1:6]
        respon = {"tu juego es" : df_ML_reduced[df_ML_reduced["item_id"]==id_item].iloc[0,1],
                "recomendaciones": []}
        for i in jueguito:
            respon["recomendaciones"].append(df_ML_reduced.iloc[i[0]].title)
        
        return respon
    except IndexError:
        print("buscando recomendaciones, por favor espere")
        new_game = df_ML[df_ML["item_id"] == id_item]
        df_combined = pd.concat([df_ML,new_game])
        vectors = cv.fit_transform(df_combined["tags"]).toarray()
        similarity=cosine_similarity(vectors)
        idx = df_combined[df_combined["item_id"] == id_item].index[0]
        distances = similarity[idx]   
        jueguito = sorted(list(enumerate(distances)), reverse=True, key= lambda x:x[1])[1:6]
        respon = {"tu juego es" : df_combined[df_combined["item_id"]==id_item].iloc[0,1],
        "recomendaciones": []}
        for i in jueguito:
            respon["recomendaciones"].append(df_combined.iloc[i[0]].title)
            
        return respon
            


Prueba con id de juego que no estaba dentro del df que se uso para hacer el modelo ya que este se hizo unicamente con los primeros 2000 juegos basados en un score.

In [8]:
recommend(761140.0) 

buscando recomendaciones, por favor espere


{'tu juego es': 'Lost Summoner Kitty',
 'recomendaciones': ['Wildlife Park 2 - Fantasy',
  'Mad Games Tycoon',
  'Wildlife Park 2',
  'BalanCity',
  'Lost Summoner Kitty']}

### Pruebas de modelos con *Counter-Strike*

En las siguientes lineas voy a probar hiperparametros. unicamente voy a cambiar el "max_features" que hace referencia a la cantidad de tags que estaria tomando de la tabla df_itemsxitems

Lo que estoy buscando especificamente es que no recomiende con una perfeccion del 100% ya que mientras mas potente sea el modelo, mas esperables son las recomendaciones.

En los casos en que teniamos un elevado nivel de features y mas juegos en nuestra base de datos, mas esperables son los resultados.

En mi modelo no pretendo lograr una recomendacion perfecta pues, si pido una recomendacion de un juego con base a algo que ya he jugado, lo mas probable es que ya conosca de sus secuelas o otras adaptaciones. Una recomendacion a mi parecer debe ser lo suficientemente cercana al juego original como para manterer ciertas "features" que hacen al genero o tipo de jugabilidad, pero no tanto como para que sea redundante (como sucede en la primer muestra)


Por este motivo elijo entrenar a mi con esos hiperparametros especificos. 

Tambien debo aclarar que elegi este juego por que era el primero de la lista por su numero de id y por que es casi conocido por todos. Esta misma logica dudo que se aplique para el total de los juegos de nuestra base de datos.

In [14]:
recommend(10) #max_features = 2000. 5000 juegos en la base de datos

{'tu juego es': 'Counter-Strike',
 'recomendaciones': ["Tom Clancy's Rainbow Six® Siege - Ultra HD Texture Pack",
  'Empires Mod',
  'Deathmatch Classic',
  'Day of Defeat',
  'ARMA: Gold Edition']}

In [15]:
recommend(10) #max_features = 500  2000 juegos en la base de datos

{'tu juego es': 'Counter-Strike',
 'recomendaciones': ["Tom Clancy's Rainbow Six® Siege - Ultra HD Texture Pack",
  'Empires Mod',
  'Deathmatch Classic',
  'Day of Defeat',
  'ARMA: Gold Edition']}

In [16]:
recommend(10) #max_features = 50   2000 juegos en la base de datos

{'tu juego es': 'Counter-Strike',
 'recomendaciones': ["Tom Clancy's Rainbow Six® Siege - Ultra HD Texture Pack",
  'Empires Mod',
  'Deathmatch Classic',
  'Day of Defeat',
  'ARMA: Gold Edition']}

In [12]:
recommend(10) #max_feautures = 40   2000 juegos en la base de datos

{'tu juego es': 'Counter-Strike',
 'recomendaciones': ["Tom Clancy's Rainbow Six® Siege - Ultra HD Texture Pack",
  'Empires Mod',
  'Deathmatch Classic',
  'Day of Defeat',
  'ARMA: Gold Edition']}

In [13]:
recommend(10) #max_features = 45   2000 juegos en la base de datos

{'tu juego es': 'Counter-Strike',
 'recomendaciones': ["Tom Clancy's Rainbow Six® Siege - Ultra HD Texture Pack",
  'Empires Mod',
  'Deathmatch Classic',
  'Day of Defeat',
  'ARMA: Gold Edition']}